In [2]:
import selenium.webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Kkma
import os
from sklearn.decomposition import LatentDirichletAllocation
kkma = Kkma()


# 1 웹 크롤링 - 약 3분 소요 
> ## 해당 데이터 약 2000개 
리스트에 웹 크롤링 "{230801}_daily_commu.csv" 로 저장 및 리스트로 자동 변환
# 2 해당 데이터 저장 및 변환 
> ### 실행 전 해당 커뮤니티 데이터 파일명 변경 필요!
# 3 Countvectorizer, Latent Dirichlet Allocation 학습 
> ### 현재 학습 시간 5초 미만, grid search 시   
{'countvectorizer__max_df': 0.3,  
 'countvectorizer__ngram_range': (3, 3),  
 'latentdirichletallocation__n_components': 4}  
라는 결과 
---

# 전체 코드 

In [7]:
dc_list = []
bbomppu_list = []
fm_list = []
tq_list = []
eto_list = []
# 옵션 생성
options = wb.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")
# driver 실행
driver = wb.Chrome(options=options)

# dc 수집
driver.get("https://gall.dcinside.com/board/lists/?id=dcbest")
# 100개씩 보기 
driver.find_element(By.CSS_SELECTOR, "#container > section.left_content > article:nth-child(3) > div.list_array_option.clear > div.right_box > div > div.select_box.array_num > div > a").click()
driver.find_element(By.CSS_SELECTOR, "#listSizeLayer > li:nth-child(3) > a").click()
soup = bs(driver.page_source, "lxml")
for j in range(2, 5):
    for i in range(3, 102):
        title = soup.select(f"article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child({i}) > td.gall_tit.ub-word > a:nth-child(1)")
        dc_list.append((title[0].text)[5:])

# bbomppu 수집
driver.get("https://www.ppomppu.co.kr/hot.php?category=2")
# 그냥 바로 추출 
soup = bs(driver.page_source, "lxml")
# 첫 1-10 까지 
for j in range(2, 9):
    for i in range(6, 26):
        title = soup.select(f"table.board_table > tbody > tr:nth-child({i}) > td:nth-child(4) > a")
        bbomppu_list.append(title[0].text)
    driver.find_element(By.CSS_SELECTOR, f"#page_list > font > a:nth-child({j})").click()
    time.sleep(0.5)
# 다음으로 넘기기 (11page로)
driver.find_element(By.CSS_SELECTOR, "#page_list > font > a.page_next").click()
time.sleep(0.5)

# 11페이지 부터 추출 
for k in range(5):
    for j in range(3, 7):
        for i in range(6, 26):
            title = soup.select(f"table.board_table > tbody > tr:nth-child({i}) > td:nth-child(4) > a")
            bbomppu_list.append(title[0].text)
        driver.find_element(By.CSS_SELECTOR, f"#page_list > font > a:nth-child({j})").click() 
        time.sleep(0.5)

# fm 수집
driver.get("https://www.fmkorea.com/index.php?mid=best&listStyle=list&page=1")
# 8페이지까지만 가져온 후 
for j in range(9, 16):
    for i in range(2, 22):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_189545458_0 > div > table > tbody > tr:nth-child({i}) > td.title.hotdeal_var8 > a.hx")
        fm_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"#bd_189545458_0 > div > form > fieldset > a:nth-child({j})").click()
    time.sleep(0.5)
# fm 페이지 특징: 8페이지 부터는 계속 위치 고정 
for j in range(16, 31):
    for i in range(2, 22):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_189545458_0 > div > table > tbody > tr:nth-child({i}) > td.title.hotdeal_var8 > a.hx")
        fm_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, "#bd_189545458_0 > div > form > fieldset > a:nth-child(16)").click() 
    time.sleep(0.5)
    
# the qoo 수집 
driver.get("https://theqoo.net/hot?page=1&filter_mode=normal")
# the qoo도 7페이지 부터 페이지 순서 고정 6까지 가져오기 
for j in range(4, 10):
    for i in range(11, 31):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_2842877099_0 > div > table > tbody > tr:nth-child({i}) > td.title > a:nth-child(1)")
        tq_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"#bd_2842877099_0 > div > form > ul > li:nth-child({j}) > a").click()
    time.sleep(0.5)
# the qoo 페이지 특징: 8페이지 부터는 계속 위치 고정 
for j in range(11, 14):
    for i in range(11, 31):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_2842877099_0 > div > table > tbody > tr:nth-child({i}) > td.title > a:nth-child(1)")
        tq_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"#bd_2842877099_0 > div > form > ul > li:nth-child(10) > a").click()
    time.sleep(0.5)
    
# etoland 추출 
driver.get("https://etoland.co.kr/pages/hit.php")
# 1~2 페이지 추출 
for j in range(2, 3):
    for i in range(5, 65):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"body > main > section > div.board_hit_wrap > ul > li:nth-child({i}) > div.subject > a.sub_link > span.subject_txt")
        eto_list.append((title[0].text).strip("\n").strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"body > main > section > div.board_hit_wrap > div.write_pages_wrap > div.write_pages > a:nth-child(2)").click()
    time.sleep(0.5)
    

for j in range(4, 7):
    for i in range(5, 65):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"body > main > section > div.board_hit_wrap > ul > li:nth-child({i}) > div.subject > a.sub_link > span.subject_txt")
        eto_list.append((title[0].text).strip("\n").strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"body > main > section > div.board_hit_wrap > div.write_pages_wrap > div.write_pages > a:nth-child({j})").click()
    time.sleep(0.5)

# driver 종료
driver.quit()


# 모아진 데이터 불용어 처리 
daily_all_commu = pd.concat([pd.DataFrame(dc_list), pd.DataFrame(bbomppu_list), pd.DataFrame(fm_list), pd.DataFrame(eto_list), pd.DataFrame(tq_list)])
daily_all_commu.columns = ["content"]
daily_all_commu.to_csv("230801_daily_commu.csv", encoding='utf-8-sig')
daily_all_commu = daily_all_commu.loc[:, "content"].to_list()

# 불용어 지정. 많으면 많을 수록 좋을 듯.. 
stopwords = ['-', 'jpg', 'ㅋ', '[ㅇㅎ]', "gif", '스압', 'ㅇㅎ', 'ㄷ', "ㅎ", ';', 'twt', 'blind', 'pann', 'gisa', '★', '☆', 'ㅠ', 'ㅜ', "manhwa", "mp4", "후방", "레전드", "주의", "JPG", 'webp', "boja"]

# 불용어가 제거된 데이터를 저장할 새로운 리스트
filtered_list = []

for item in daily_all_commu:
    for stopword in stopwords:
        item = item.replace(stopword, '')
    filtered_list.append(item.strip())
# 총 데이터 갯수 확인
print(len(filtered_list))

# 불용어 지정. 많으면 많을 수록 좋을 듯.. 
stopwords = ['-', 'jpg', 'ㅋ', '[ㅇㅎ]', "gif", '스압', 'ㅇㅎ', 'ㄷ', "ㅎ", ';', 'twt', 'blind', 'pann', 'gisa', '★', '☆', 'ㅠ', 'ㅜ', "manhwa", "mp4", "후방", "레전드", "주의", "JPG", 'webp', "boja"]

# 불용어가 제거된 데이터를 저장할 새로운 리스트
filtered_list = []

for item in daily_all_commu:
    for stopword in stopwords:
        item = item.replace(stopword, '')
    filtered_list.append(item.strip())
# 총 데이터 갯수 확인
print(len(filtered_list))
filtered_list


# vectorizer와 lda모델 학습
vectorizer = CountVectorizer(max_df = 0.3, min_df = 1, stop_words="english", ngram_range=(3, 3))
X = vectorizer.fit_transform(filtered_list)
#LDA 모델 학습
num_topics = 4 # 원하는 토픽의 수 설정
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state= 42)
lda_model.fit(X)

# display 토픽 보여주기 
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print("\nTopic #", topic_idx+1)
        # Topic별로 1000개의 단어들(features)중에서 높은 값 순으로 정렬 후 index를 반환해줌
        # argsort()는 default가 오름차순(1, 2, 3,..) 그래서[::,-1]로 내림차순으로 바꾸기
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]
        
        # CountVectorizer 함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feature(단어들) 볼 수 있음.
        # 이 벡터화시킨 단어들(features)은 숫자-알파벳순으로 정렬되며, 단어들 순서는 fit_transform시키고 난 이후에도 동일! 
        # "문자열".join함수로 특정 문자열 사이에 끼고 문자열 합쳐줄 수 있음
        feature_concat = " ".join([str(feature_names[i])+" "for i in top_idx[:4]])
        
        print(feature_concat)
feature_names = vectorizer.get_feature_names_out()
display_topic_words(lda_model, feature_names, 15)

# 전체 코드 끝
### 약 4분 내외로 소요 및 
### 실행 전 해당 커뮤니티 데이터 파일명 변경 필요!
---
---
---
---

In [19]:
daily_all_commu = pd.concat([pd.DataFrame(dc_list), pd.DataFrame(bbomppu_list), pd.DataFrame(fm_list), pd.DataFrame(eto_list), pd.DataFrame(tq_list)])
daily_all_commu.columns = ["content"]
daily_all_commu.to_csv("230801_daily_commu.csv", encoding='utf-8-sig')
daily_all_commu = daily_all_commu.loc[:, "content"].to_list()



297 540 440 180 240


In [18]:
daily_all_commu = pd.read_csv("230801_daily_commu.csv", encoding= 'utf-8-sig')
daily_all_commu = daily_all_commu.loc[:, "content"].to_list()

# 불용어 지정. 많으면 많을 수록 좋을 듯.. 
stopwords = ['-', 'jpg', 'ㅋ', '[ㅇㅎ]', "gif", '스압', 'ㅇㅎ', 'ㄷ', "ㅎ", ';', 'twt', 'blind', 'pann', 'gisa', '★', '☆', 'ㅠ', 'ㅜ', "manhwa", "mp4", "후방", "레전드", "주의", "JPG", 'webp', "boja"]

# 불용어가 제거된 데이터를 저장할 새로운 리스트
filtered_list = []

for item in daily_all_commu:
    for stopword in stopwords:
        item = item.replace(stopword, '')
    filtered_list.append(item.strip())
# 총 데이터 갯수 확인
print(len(filtered_list))
filtered_list


# vectorizer와 lda모델 학습
vectorizer = CountVectorizer(max_df = 0.3, min_df = 1, stop_words="english", ngram_range=(3, 3))
X = vectorizer.fit_transform(filtered_list)
#LDA 모델 학습
num_topics = 4 # 원하는 토픽의 수 설정
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state= 42)
lda_model.fit(X)

# display 토픽 보여주기 
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print("\nTopic #", topic_idx+1)
        # Topic별로 1000개의 단어들(features)중에서 높은 값 순으로 정렬 후 index를 반환해줌
        # argsort()는 default가 오름차순(1, 2, 3,..) 그래서[::,-1]로 내림차순으로 바꾸기
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]
        
        # CountVectorizer 함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feature(단어들) 볼 수 있음.
        # 이 벡터화시킨 단어들(features)은 숫자-알파벳순으로 정렬되며, 단어들 순서는 fit_transform시키고 난 이후에도 동일! 
        # "문자열".join함수로 특정 문자열 사이에 끼고 문자열 합쳐줄 수 있음
        feature_concat = " ".join([str(feature_names[i])+" "for i in top_idx[:4]])
        
        print(feature_concat)
feature_names = vectorizer.get_feature_names_out()
display_topic_words(lda_model, feature_names, 15)

1697


["박보영이 주인공인 '너의 결혼식'은 남성성 거세 영화라고 말한 유튜버",
 'oth 개발일지 1#) 액션은 학생의 무덤',
 '싱글벙글 건강한 할머니',
 '[] 재난 3부작 도쿄 성지순례 후기',
 '장산범이 초등학교 다니는 만화.',
 '커여운 흰제비갈매기의 습성',
 '파워퍼프걸 그림 모음',
 '싱글벙글 43850번 버스',
 '크시 건담 부분도색 + 지리워크데칼(사진많음)',
 '아마 세계최초 언럭키찬나 부화',
 '감자국 휴가 1부',
 '()2023년 상반기 인갤 인기 게임 목록',
 '그림체 변화 과정',
 '싱글벙글 독 짓는 늙은이 실사 영화',
 '풍경은 도박하는 기분이기도함',
 '수집가들의 세계  번외2. 닌텐도 게임기 한국 정발의 역사',
 '싱글벙글 한중일 변화',
 ') 칵린이 여러 술들 리뷰',
 '단편 떨어진 4컷만화 업로드해봅니다',
 '싱글벙글 문명의 충돌.',
 '수정 후 재업?) 나만의 작은 위스키통 같은 건 없을까?',
 '() 북한 전승절 열병식 충격으로 TS해본.',
 '초전도체 이석배 스승 최동식 교수',
 'clux들고 다녀온 거제 여행 사진',
 '타타리가미) 러시아군 훈련 및 러시아군의 붕괴에 대해',
 '싱글벙글 디씨식 반박.',
 '(정보) 미프 팁 모음',
 'LK99와 타임머신의 공통점...',
 '강풀, 서장훈 대화.',
 '바깥은 그야말로 여름이었다',
 '깍두기 먹는다고 혼내는 일본의 어느 방송',
 '인터넷에 왜곡돼서 돌아다니는 캐릭터 밈.',
 '가난한 소시민 설정을 기가 막히게 잘 쓴 히어로..',
 '국내 유명 호텔들 짜증나는 부분',
 '싱글벙글 보석에 대해 ARABOZA',
 '폴드/플립5 카메라 관련 신규 기능들',
 '세계 성평등지수에서 대한민국 순위가 낮은이유',
 '상온 초전도체 성공시 가장 큰 문제점들 ㅗㅑ',
 '요즘 에어컨 기사들 수준이하 많다',
 '만화가 김성모, "남편은 가장으로서 대접 받아야 한다"',
 '오늘자 곽튜브..... 입국심사 오래 걸린 이유.....',

In [74]:
vectorizer = CountVectorizer(max_df = 0.3, min_df = 1, stop_words="english", ngram_range=(3, 3))
X = vectorizer.fit_transform(filtered_list)
#LDA 모델 학습
num_topics = 4 # 원하는 토픽의 수 설정
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state= 42)
lda_model.fit(X)

LatentDirichletAllocation(n_components=4, random_state=42)

In [73]:
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print("\nTopic #", topic_idx+1)
        # Topic별로 1000개의 단어들(features)중에서 높은 값 순으로 정렬 후 index를 반환해줌
        # argsort()는 default가 오름차순(1, 2, 3,..) 그래서[::,-1]로 내림차순으로 바꾸기
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]
        
        # CountVectorizer 함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feature(단어들) 볼 수 있음.
        # 이 벡터화시킨 단어들(features)은 숫자-알파벳순으로 정렬되며, 단어들 순서는 fit_transform시키고 난 이후에도 동일! 
        # "문자열".join함수로 특정 문자열 사이에 끼고 문자열 합쳐줄 수 있음
        feature_concat = " ".join([str(feature_names[i])+" "for i in top_idx[:4]])
        
        print(feature_concat)
feature_names = vectorizer.get_feature_names_out()
display_topic_words(lda_model, feature_names, 15)


Topic # 1
샤워하면서 소변 보세요  국제 광고상 수상작  소변 보세요 국제  보세요 국제 광고상 

Topic # 2
차에 낙서를 했다  딸내미가 유성매직으로 차에  유성매직으로 차에 낙서를  9살 딸내미가 유성매직으로 

Topic # 3
선택하지 않고 카레맛  카레를 선택하지 않고  유시민 똥맛나는 카레를  똥맛나는 카레를 선택하지 

Topic # 4
파도 계속 나오는  계속 나오는 충북도지사  파도 파도 계속  나오는 충북도지사 근황 

Topic # 5
인식문제로 발생한 사고  징역 8년 확정  이슬비 징역 8년  여신 이슬비 징역 


# GridSearchCV(교차검증 실습)

In [75]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


In [64]:
lda_pipe = make_pipeline(CountVectorizer(), LatentDirichletAllocation())

param_grid = {
    'countvectorizer__max_df': [0.3, 0.4, 0.5],
    'countvectorizer__ngram_range': [(1, 2), (2, 2), (2, 3), (3, 3), (3, 4)],
    'latentdirichletallocation__n_components': [4, 5, 6,7, 8]
}

## data split - split도 필요없음. 그냥 random으로 나누기

In [56]:
import random
random.shuffle(filtered_list)

# 전체 데이터 개수
total_samples = len(filtered_list)

# 80%는 훈련 세트, 20%는 테스트 세트로 나눔
train_size = int(0.8 * total_samples)

# 데이터를 훈련 세트와 테스트 세트로 나눔
X_train = filtered_list[:train_size]
X_test = filtered_list[train_size:]

# 라벨링이 없으므로 y_train, y_test는 None으로 설정
y_train = None
y_test = None

In [65]:

grid = GridSearchCV(lda_pipe, param_grid, cv=3)

In [66]:
grid.fit(X_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('latentdirichletallocation',
                                        LatentDirichletAllocation())]),
             param_grid={'countvectorizer__max_df': [0.3, 0.4, 0.5],
                         'countvectorizer__ngram_range': [(1, 2), (2, 2),
                                                          (2, 3), (3, 3),
                                                          (3, 4)],
                         'latentdirichletallocation__n_components': [4, 5, 6, 7,
                                                                     8]})

# grid search 결과 - max_df(0.3)/ngram(3,3)/n_components(4) 

In [67]:
grid.best_params_

{'countvectorizer__max_df': 0.3,
 'countvectorizer__ngram_range': (3, 3),
 'latentdirichletallocation__n_components': 4}

-5969.936818868628

In [ ]:
driver = wb.Chrome()
driver.get("https://gall.dcinside.com/board/lists/?id=dcbest")

# 100개씩 보기 
driver.find_element(By.CSS_SELECTOR, "#container > section.left_content > article:nth-child(3) > div.list_array_option.clear > div.right_box > div > div.select_box.array_num > div > a").click()
driver.find_element(By.CSS_SELECTOR, "#listSizeLayer > li:nth-child(3) > a").click()
soup = bs(driver.page_source, "lxml")
for j in range(2, 5):
    for i in range(3, 102):
        title = soup.select(f"article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child({i}) > td.gall_tit.ub-word > a:nth-child(1)")
        dc_list.append((title[0].text)[5:])
#     # 다음페이지 
#     driver.find_element(By.CSS_SELECTOR, f"article:nth-child(3) > div.bottom_paging_wrap > div.bottom_paging_box.iconpaging > a:nth-child({j})").click()
# #16페이지 부터 또 하기 
# driver.find_element(By.CSS_SELECTOR, "article:nth-child(3) > div.bottom_paging_wrap > div.bottom_paging_box.iconpaging > a.sp_pagingicon.page_next").click()
# for j in range(4, 18):
#     for i in range(3, 102):
#         title = soup.select(f"article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child({i}) > td.gall_tit.ub-word > a:nth-child(1)")
#         dc_list.append((title[0].text)[5:])
#     driver.find_element(By.CSS_SELECTOR, f"article:nth-child(3) > div.bottom_paging_wrap > div.bottom_paging_box.iconpaging > a:nth-child({j})").click()

# 학습데이터 가져오기 - bbomppu 

In [268]:
driver = wb.Chrome()
driver.get("https://www.ppomppu.co.kr/hot.php?category=2")
# 그냥 바로 추출 
soup = bs(driver.page_source, "lxml")
# 첫 1-10 까지 
for j in range(2, 11):
    for i in range(6, 26):
        title = soup.select(f"table.board_table > tbody > tr:nth-child({i}) > td:nth-child(4) > a")
        bbomppu_list.append(title[0].text)
    driver.find_element(By.CSS_SELECTOR, f"#page_list > font > a:nth-child({j})").click()
    time.sleep(0.5)
# 다음으로 넘기기 (11page로)
driver.find_element(By.CSS_SELECTOR, "#page_list > font > a.page_next").click()
time.sleep(0.5)

# 11페이지 부터 추출 
for k in range(5):
    for j in range(3, 7):
        for i in range(6, 26):
            title = soup.select(f"table.board_table > tbody > tr:nth-child({i}) > td:nth-child(4) > a")
            bbomppu_list.append(title[0].text)
        driver.find_element(By.CSS_SELECTOR, f"#page_list > font > a:nth-child({j})").click() 
        time.sleep(0.5)


# 학습 데이터 가져오기 - fm

In [306]:
driver = wb.Chrome()
driver.get("https://www.fmkorea.com/index.php?mid=best&listStyle=list&page=1")
# 8페이지까지만 가져온 후 
for j in range(9, 16):
    for i in range(2, 22):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_189545458_0 > div > table > tbody > tr:nth-child({i}) > td.title.hotdeal_var8 > a.hx")
        fm_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"#bd_189545458_0 > div > form > fieldset > a:nth-child({j})").click()
    time.sleep(0.5)
# fm 페이지 특징: 8페이지 부터는 계속 위치 고정 
for j in range(16, 45):
    for i in range(2, 22):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_189545458_0 > div > table > tbody > tr:nth-child({i}) > td.title.hotdeal_var8 > a.hx")
        fm_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, "#bd_189545458_0 > div > form > fieldset > a:nth-child(16)").click() 
    time.sleep(0.5)


---
# 학습 데이터 가져오기 - the qoo 

In [317]:
driver = wb.Chrome()
driver.get("https://theqoo.net/hot?page=1&filter_mode=normal")


soup = bs(driver.page_source, "lxml")
# the qoo도 7페이지 부터 페이지 순서 고정 6까지 가져오기 
for j in range(4, 10):
    for i in range(11, 31):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_2842877099_0 > div > table > tbody > tr:nth-child({i}) > td.title > a:nth-child(1)")
        tq_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"#bd_2842877099_0 > div > form > ul > li:nth-child({j}) > a").click()
    time.sleep(0.5)
# the qoo 페이지 특징: 8페이지 부터는 계속 위치 고정 
for j in range(11, 14):
    for i in range(11, 31):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"#bd_2842877099_0 > div > table > tbody > tr:nth-child({i}) > td.title > a:nth-child(1)")
        tq_list.append((title[0].text).strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"#bd_2842877099_0 > div > form > ul > li:nth-child(10) > a").click()
    time.sleep(0.5)


# 데이터 수집하기 - eToLAND 

In [11]:
driver = wb.Chrome()
driver.get("https://etoland.co.kr/pages/hit.php")

# 1~2 페이지 추출 
for j in range(2, 3):
    for i in range(5, 65):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"body > main > section > div.board_hit_wrap > ul > li:nth-child({i}) > div.subject > a.sub_link > span.subject_txt")
        eto_list.append((title[0].text).strip("\n").strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"body > main > section > div.board_hit_wrap > div.write_pages_wrap > div.write_pages > a:nth-child(2)").click()
    time.sleep(0.5)
    

for j in range(4, 7):
    for i in range(5, 65):
        soup = bs(driver.page_source, "lxml")
        title = soup.select(f"body > main > section > div.board_hit_wrap > ul > li:nth-child({i}) > div.subject > a.sub_link > span.subject_txt")
        eto_list.append((title[0].text).strip("\n").strip("\t"))
    driver.find_element(By.CSS_SELECTOR, f"body > main > section > div.board_hit_wrap > div.write_pages_wrap > div.write_pages > a:nth-child({j})").click()
    time.sleep(0.5)
    
# #페이지 넘어가기 
# driver.find_element(By.CSS_SELECTOR, "div.board_hit_wrap > div.write_pages_wrap > div.write_pages > a:nth-child(12)").click()
# time.sleep(0.5)
# # 11페이지 부터 ~ 
# for k in range(5):
#     for j in range(4, 13):
#         for i in range(5, 65):
#             soup = bs(driver.page_source, "lxml")
#             title = soup.select(f"body > main > section > div.board_hit_wrap > ul > li:nth-child({i}) > div.subject > a.sub_link > span.subject_txt")
#             eto_list.append((title[0].text).strip("\n").strip("\t"))
#         driver.find_element(By.CSS_SELECTOR, f"body > main > section > div.board_hit_wrap > div.write_pages_wrap > div.write_pages > a:nth-child({j})").click()
#         time.sleep(0.5)
#     driver.find_element(By.CSS_SELECTOR, "body > main > section > div.board_hit_wrap > div.write_pages_wrap > div.write_pages > a:nth-child(13)").click() 
#     time.sleep(0.5)
